In [61]:

# TODO change the code into function
# TODO add looping functionarity to be able to iterate multiple links
# TODO figure out how to deal with exceptions 
# TODO analyze the data and perform visualization

In [62]:
import requests
import praw
from praw.models import MoreComments 
import pandas as pd
from collections import defaultdict
from bs4 import BeautifulSoup
import json

In [63]:
%load_ext blackcellmagic

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [64]:
def main():
    """
    docstring
    """
    json_file = "reddit_api_config.json"
    json_key = "reddit_api"
    url = "https://www.reddit.com/r/Accounting/comments/hb5u1h/pwc_2020_compensation_thread/"
    reddit = authenticate(json_file=json_file, json_key=json_key)
    submission = get_submission(reddit=reddit, url=url)
    data_dict = get_data(submission)
    file_name = get_filename(submission=submission)
    save_dict(data_dict,file_name)
    

In [65]:
def authenticate(json_file, json_key):

    # open the json file and load the data as dictionary
    with open(json_file) as f:
        data = json.load(f)
    # look into the schema of json_key in json_file
    user_values = data[json_key]

    # initiate the session
    reddit = praw.Reddit(
        client_id=user_values["client_id"],
        client_secret=user_values["client_secret"],
        user_agent=user_values["user_agent"],
    )
    return reddit

In [66]:
def check_authentication(reddit):
    # Test if the read only instance was created properly 
    print(reddit.read_only)

In [67]:
def get_submission(reddit, url):
    """
    docstring
    """
    # Create submission object based on the url 
    submission = reddit.submission(url=url)
    return submission


In [68]:
def get_filename(submission):
    """
    docstring
    """
    file_name = submission.title.replace(" ", "_")
    file_name = f"{file_name}.csv"
    return file_name


In [69]:
def get_category(submission):
    # obtaining the body of the OP's text in html format
    submission_text = submission.selftext_html
    # create soup object 
    soup = BeautifulSoup(submission_text, 'html.parser')
    # the list is always in ordered list, or ol. hence find that 
    category_list = soup.find("ol")
    # we category_list.text will provide the str of category, hence split to make a list 
    category_list = category_list.text.split("\n")
    # remove the empty elements in the list 
    category_list = list(filter(None,category_list))
    # numbering for the category list
    return category_list


In [70]:
def get_number (category_list):
    """
    docstring
    """
    return range(1, len(category_list)+1)
# print(lookup_dict)



In [71]:
def get_lookup_dict(category_list):
    """
    docstring
    """
    # create lookup dictionary to identify category based on the number 
    return dict(zip(num_list, category_list))


In [72]:
def get_check_dict(num_list):
    """
    docstring
    """
    # this dictionary will keep track of which items are responded
    return {i:False for i in num_list}
    

In [73]:

def get_data(submission):
    
    category_list = get_category(submission=submission)
    num_list = get_number(category_list=category_list)
    lookup_dict = get_lookup_dict(category_list=category_list)
    check_dict = get_check_dict(num_list=num_list)
    
    # obtain the submission comments object 
    submission_dict = defaultdict(list)
    comments = submission.comments
    # Need to call replace_more in order to avoid the Morecomments error
    comments.replace_more(limit=None)
    # for each comment in the comments object
    for comment in comments: 
        print(f"worked on the {comment.body}")
        # obtain the comment body 
        content = comment.body
        # split the contents line by line  
        content = content.split("\n")
        # remove empty strings from the list 
        content = list(filter(None, content))
        # only select line that start with the numbers 
        content = [i for i in content if i[0].isnumeric()]
        # if the list is blank 
        if not content: 
            #skip that item
            continue
        # create list of numbers from dict keys
        key_list = str(list(lookup_dict.keys()))
        # for each line in the content, or the comment 
        for line in content:
            # the number is always the first character 
            heading_number = float(line[0])
            # obtain the category using the look up table 
            if check_dict[heading_number] == True: 
                continue
            category = lookup_dict[heading_number]
            check_dict[heading_number] = True
            # append the response based on the category key 
            submission_dict[category].append(line.strip()[2:].strip())
            # print(len(submission_dict[category]))

            # print(f"worked on the {line}")
        print(num_list)
        for number in num_list:
            if check_dict[number]:
                check_dict[number]=False
                # print(f"did respond {number}")
                # continue
            else:
                print(number)
                category = lookup_dict[number]
                # print("category")
                # print(f"before {submission_dict[category]}")
                submission_dict[category].append("")
                # print(f"after {submission_dict[category]}")
                check_dict[number]=False
                # print(f"did not respond {number}")
                # continue
    return submission_dict
        
        # we need to be careful because some people do not post response for all the question. Therefore, if they skip one or 

In [74]:
def save_data(data_dict, file_name):
    df = pd.DataFrame(data_dict)
    df.to_csv(file_name)

In [75]:
%reset -f
# The above magic command reset the variable, -f denote no user confirmation 